In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

import pickle
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [0]:
from keras import Sequential
from keras.layers import Dense

In [0]:
def Create_NN_Model(No_Features=300, No_Hidden_Layers=2, No_Hidden_Neurons=150, 
                    Hidden_Activation ="relu", No_OP_Neurons=1, 
                    Output_Activation="sigmoid", Kernel_Initializer="random_normal",
                    Optimizer="adam", Loss='binary_crossentropy', Metrics =['accuracy']):
  
  classifier = Sequential()

  ## Input Layer
  classifier.add(Dense(No_Hidden_Neurons, activation=Hidden_Activation, 
                       kernel_initializer=Kernel_Initializer, input_dim=No_Features))
  
  ## Hidden layers
  for i in range(No_Hidden_Layers):
    classifier.add(Dense(No_Hidden_Neurons, activation=Hidden_Activation, 
                         kernel_initializer=Kernel_Initializer))
    
  ## Output Layer
  classifier.add(Dense(No_OP_Neurons, activation=Output_Activation, 
                       kernel_initializer=Kernel_Initializer))
  
  classifier.compile(optimizer =Optimizer, loss=Loss, metrics = Metrics)

  return classifier
  

In [0]:
def Train_NN(NN_classifier, train_data, feature_list=[], Batch_Size=50, Epochs=100):

  train_data.dropna()
  train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
  train_data['Label'] = pd.to_numeric(train_data['Label'], errors='coerce')
  train_data = train_data.dropna(subset=['Label'])
  
  train_features = train_data[feature_list]    
  train_labels = train_data["Label"]
  train_labels = train_labels.astype('int')

  NN_classifier.fit(train_features,train_labels, batch_size=Batch_Size, epochs=Epochs)

  eval_model=NN_classifier.evaluate(train_features, train_labels)
  print("Loss: ", eval_model[0])
  print("Accuracy of the model: ", eval_model[1])

  return NN_classifier


In [0]:
## Store trained model in a file to reuse in other codes without training again on same data

def Store_Trained_NN(NN_obj, Filepath):
  
  with open(Filepath, "wb") as file:
    pickle.dump(NN_obj, file)

In [0]:
## Load stored trained model and returns random forest model object

def Load_Trained_NN(Filepath):
  
  with open(Filepath, "rb") as file:
    NN_obj = pickle.load(file)

  return NN_obj

In [0]:
def Evaluate_NN(test_data, NN_Model_FilePath, feature_list=[], threshold=0.5):
  
  test_data.dropna()
  test_data = pd.DataFrame(np.nan_to_num(np.array(test_data)),  columns = test_data.columns)
  test_data['Label'] = pd.to_numeric(test_data['Label'], errors='coerce')
  test_data = test_data.dropna(subset=['Label'])

  test_features = test_data[feature_list]
  test_labels = test_data["Label"]
  test_labels = test_labels.astype('int')

  NN_obj = Load_Trained_NN(NN_Model_FilePath) 
  predictions = NN_obj.predict(test_features)
  predictions_list = [int(p[0]) for p in predictions]
  
  for i in range(len(predictions_list)):
    if predictions_list[i] >= threshold:
      predictions_list[i] = 1
    else:
      predictions_list[i] = 0
  
  errors = abs(predictions_list - test_labels)

  # Calculate mean absolute error (MAE)
  MAE = round(np.mean(errors), 2)
  
  ## Confusion Matrix and Classification Report
  Confusion_Matrix = confusion_matrix(test_labels,predictions_list)
  Report = classification_report(test_labels,predictions_list)
  
  return MAE, Confusion_Matrix, Report
  

In [8]:
## WORD2VEC EMBEDDINGS

Column_List = [ "Caption"]
Vector_Size = 300
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.append("Label")

Train_Embedding_FilePath = "/content/TrainData_Word2Vec_Embeddings.csv"
Test_Embedding_FilePath = "/content/TestData_Word2Vec_Embeddings.csv"
NN_Model_FilePath = "/content/NN_Word2Vec_Train_Model.pkl"

train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)

## Training Phase
NN_Classifier = Create_NN_Model()
NN_obj = Train_NN(NN_Classifier, train_data, Embedding_Cols)
Store_Trained_NN(NN_obj, NN_Model_FilePath)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





5824/5824 [==============================] - 1s 198us/step - loss: 0.4709 - acc: 0.7701
Epoch 2/100
5824/5824 [==============================] - 1s 154us/step - loss: 0.3474 - acc: 0.8424
Epoch 3/100
5824/5824 [==============================] - 1s 156us/step - loss: 0.3031 - acc: 0.8645
Epoch 4/100
5824/5824 [==============================] - 1s 150us/step - loss: 0.2841 - acc: 0.8740
Epoch 5/100
5824/5824 [==============================] - 1s 156us/step - loss: 0.2561 - acc: 0.8906
Epoch 6/100
5824/5824 [==============================] - 1s 153us/step - loss: 0.2326 - acc: 0.9008
Epoch 7/100
5824/5824 [==============================] - 1s 154us/step - loss: 0.2165 - acc: 0.9062
Epoch 8/100
5824/5824 [==============================] - 1s 154us/step - loss: 0.1914 - acc: 0.9186
Epoch 9/100
5824/5824 [==============================] - 1s 151us/step - loss: 0.1762 - acc: 

In [9]:
## Testing Phase
MAE, Confusion_Matrix, Report = Evaluate_NN(test_data, NN_Model_FilePath, Embedding_Cols, 0.5)

print("============ FOR WORD2VEC EMBEDDINGS ============")

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)

============ FOR WORD2VEC EMBEDDINGS ============
MEAN ABSOLUTE ERROR:  0.37


============ CONFUSION MATRIX ===============
[[732   1]
 [536 187]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.58      1.00      0.73       733
           1       0.99      0.26      0.41       723

    accuracy                           0.63      1456
   macro avg       0.79      0.63      0.57      1456
weighted avg       0.78      0.63      0.57      1456

Accuracy:  63.11813186813187


In [10]:
## Cross Validation

train_data.dropna()
train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
train_data['Label'] = pd.to_numeric(train_data['Label'], errors='coerce')
train_data = train_data.dropna(subset=['Label'])

train_features = train_data[Embedding_Cols]    
train_labels = train_data["Label"]
train_labels = train_labels.astype('int')

# create the sklearn model for the network
model_CV = KerasClassifier(build_fn=Create_NN_Model, verbose=1)

# we choose the initializers that came at the top in our previous cross-validation!!
kernel_initializer = ['random_uniform']
batches = [64*x for x in range(1, 3)]
epochs = [50, 100, 150]
# units = [x for x in range(50, Vector_Size, 50)]
No_Hidden_Layers = [2]
optimizer = ['adam', 'rmsprop']

# No_Hidden_Neurons=units

# grid search for initializer, batch size and number of epochs
param_grid = dict(epochs=epochs, batch_size=batches, Kernel_Initializer=kernel_initializer, 
                 No_Hidden_Layers=No_Hidden_Layers, Optimizer=optimizer)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid,cv=3)
grid_result = grid.fit(train_features, train_labels)


Epoch 1/50
3882/3882 [==============================] - 1s 177us/step - loss: 0.5891 - acc: 0.6819
Epoch 2/50
3882/3882 [==============================] - 0s 94us/step - loss: 0.3899 - acc: 0.8184
Epoch 3/50
3882/3882 [==============================] - 0s 87us/step - loss: 0.3439 - acc: 0.8439
Epoch 4/50
3882/3882 [==============================] - 0s 86us/step - loss: 0.3153 - acc: 0.8534
Epoch 5/50
3882/3882 [==============================] - 0s 88us/step - loss: 0.2914 - acc: 0.8673
Epoch 6/50
3882/3882 [==============================] - 0s 90us/step - loss: 0.2708 - acc: 0.8727
Epoch 7/50
3882/3882 [==============================] - 0s 91us/step - loss: 0.2348 - acc: 0.8993
Epoch 8/50
3882/3882 [==============================] - 0s 89us/step - loss: 0.2077 - acc: 0.9132
Epoch 9/50
3882/3882 [==============================] - 0s 87us/step - loss: 0.2029 - acc: 0.9145
Epoch 10/50
3882/3882 [==============================] - 0s 85us/step - loss: 0.1885 - acc: 0.9222
Epoch 11/50
3882/

In [11]:
# print results of cross validation

print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.8593750001023431 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'rmsprop', 'batch_size': 128, 'epochs': 50}
 mean=0.8525, std=0.004631 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 50}
 mean=0.8503, std=0.004649 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 100}
 mean=0.8472, std=0.009036 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 150}
 mean=0.8486, std=0.00638 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 50}
 mean=0.8518, std=0.005884 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 100}
 mean=0.8436, std=0.003162 using {'Kernel_Initializer': 'random_uniform', 'No_Hidd

In [12]:
## GLOVE EMBEDDINGS

Column_List = [ "Caption_Tokens"]
Vector_Size = 200
Embedding_Cols = [str(i) for i in range(Vector_Size)]
Column_List.extend(Embedding_Cols)
Column_List.append("Label")

Train_Embedding_FilePath = "/content/TrainData_preTrainedGlove_Embeddings.csv"
Test_Embedding_FilePath = "/content/TestData_preTrainedGlove_Embeddings.csv"
NN_Model_FilePath = "/content/NN_Glove_Train_Model.pkl"

train_data = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)

## Training Phase
NN_Classifier = Create_NN_Model(No_Features=200, No_Hidden_Neurons=100)
NN_obj = Train_NN(NN_Classifier, train_data, Embedding_Cols)
Store_Trained_NN(NN_obj, NN_Model_FilePath)

Epoch 1/100
5824/5824 [==============================] - 3s 574us/step - loss: 0.4803 - acc: 0.7835
Epoch 2/100
5824/5824 [==============================] - 0s 86us/step - loss: 0.3475 - acc: 0.8480
Epoch 3/100
5824/5824 [==============================] - 0s 85us/step - loss: 0.3083 - acc: 0.8652
Epoch 4/100
5824/5824 [==============================] - 0s 82us/step - loss: 0.2869 - acc: 0.8783
Epoch 5/100
5824/5824 [==============================] - 0s 79us/step - loss: 0.2625 - acc: 0.8889
Epoch 6/100
5824/5824 [==============================] - 0s 84us/step - loss: 0.2377 - acc: 0.8997
Epoch 7/100
5824/5824 [==============================] - 0s 81us/step - loss: 0.2208 - acc: 0.9123
Epoch 8/100
5824/5824 [==============================] - 0s 81us/step - loss: 0.2051 - acc: 0.9174
Epoch 9/100
5824/5824 [==============================] - 0s 84us/step - loss: 0.1884 - acc: 0.9257
Epoch 10/100
5824/5824 [==============================] - 0s 84us/step - loss: 0.1706 - acc: 0.9346
Epoch 11

In [13]:
## Testing Phase
MAE, Confusion_Matrix, Report = Evaluate_NN(test_data, NN_Model_FilePath, Embedding_Cols, 0.5)

print("============ FOR GLOVE EMBEDDINGS ============")

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)

============ FOR GLOVE EMBEDDINGS ============
MEAN ABSOLUTE ERROR:  0.27


============ CONFUSION MATRIX ===============
[[686  14]
 [374 382]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.65      0.98      0.78       700
           1       0.96      0.51      0.66       756

    accuracy                           0.73      1456
   macro avg       0.81      0.74      0.72      1456
weighted avg       0.81      0.73      0.72      1456

Accuracy:  73.35164835164835


In [16]:
## Cross Validation

train_data.dropna()
train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
train_data['Label'] = pd.to_numeric(train_data['Label'], errors='coerce')
train_data = train_data.dropna(subset=['Label'])

train_features = train_data[Embedding_Cols]    
train_labels = train_data["Label"]
train_labels = train_labels.astype('int')

# create the sklearn model for the network
model_CV = KerasClassifier(build_fn=Create_NN_Model, verbose=1)

# we choose the initializers that came at the top in our previous cross-validation!!
kernel_initializer = ['random_uniform']
batches = [64*x for x in range(1, 3)]
epochs = [50, 100, 150]
# units = [x for x in range(50, Vector_Size, 50)]
No_Hidden_Layers = [2]
optimizer = ['adam', 'rmsprop']

# No_Hidden_Neurons=units

# grid search for initializer, batch size and number of epochs
param_grid = dict(No_Features=[200], epochs=epochs, batch_size=batches, Kernel_Initializer=kernel_initializer, 
                 No_Hidden_Layers=No_Hidden_Layers, Optimizer=optimizer)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid,cv=3)
grid_result = grid.fit(train_features, train_labels)

Epoch 1/50
3882/3882 [==============================] - 3s 874us/step - loss: 0.5543 - acc: 0.7102
Epoch 2/50
3882/3882 [==============================] - 0s 79us/step - loss: 0.3826 - acc: 0.8310
Epoch 3/50
3882/3882 [==============================] - 0s 82us/step - loss: 0.3309 - acc: 0.8537
Epoch 4/50
3882/3882 [==============================] - 0s 82us/step - loss: 0.2928 - acc: 0.8774
Epoch 5/50
3882/3882 [==============================] - 0s 77us/step - loss: 0.2565 - acc: 0.8954
Epoch 6/50
3882/3882 [==============================] - 0s 80us/step - loss: 0.2349 - acc: 0.9008
Epoch 7/50
3882/3882 [==============================] - 0s 81us/step - loss: 0.2086 - acc: 0.9134
Epoch 8/50
3882/3882 [==============================] - 0s 90us/step - loss: 0.1914 - acc: 0.9222
Epoch 9/50
3882/3882 [==============================] - 0s 80us/step - loss: 0.1688 - acc: 0.9320
Epoch 10/50
3882/3882 [==============================] - 0s 83us/step - loss: 0.1411 - acc: 0.9449
Epoch 11/50
3882/3

In [17]:
# print results of cross validation

print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.8619505495835955 using {'Kernel_Initializer': 'random_uniform', 'No_Features': 200, 'No_Hidden_Layers': 2, 'Optimizer': 'rmsprop', 'batch_size': 128, 'epochs': 150}
 mean=0.8546, std=0.00613 using {'Kernel_Initializer': 'random_uniform', 'No_Features': 200, 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 50}
 mean=0.8546, std=0.004388 using {'Kernel_Initializer': 'random_uniform', 'No_Features': 200, 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 100}
 mean=0.8513, std=0.006133 using {'Kernel_Initializer': 'random_uniform', 'No_Features': 200, 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 150}
 mean=0.8388, std=0.02358 using {'Kernel_Initializer': 'random_uniform', 'No_Features': 200, 'No_Hidden_Layers': 2, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 50}
 mean=0.8553, std=0.006532 using {'Kernel_Initializer': 'random_uniform', 'No_Features': 200, 'No_Hidden_Layers': 2, 'Optimizer': 'ada